# Tutorial: The Canonical HANK Model in an Open Economy Setting

We begin once more by importing packages.

In [1]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
import sequence_jacobian as sj

We again set the calibration up front.

In [2]:
calibration = {'eis': 1.,  # elasticity of intertemporal substitution
               'alpha': 0.4,  # home's foreign good preference, (1 - alpha) is the home's home bias
               'alphastar': 0.4,  # foreign's home good preference, (1 - alpha_star) is the foreign's home bias
               'eta': 1.01,  # elasticity of substitution between goods
               'gamma': 0.394,  # elasticity of foreign demand
               'markup_ss': 1.015,  # steady state markup
               'min_a': 0.,  # Minimum asset level on the grid
               'max_a': 400,  # Maximum asset level on the grid
               'n_a': 200,  # Number of asset grid points
               'rho_e': 0.92,  # persistence of idiosyncratic productivity shocks
               'sd_e': 0.57,  # standard deviation of idiosyncratic productivity shocks
               'n_e': 7,  # Number of income grid points
               'Y': 1,  # Output
               'nfa': 0,  # net foreign assets
               'rstar': 0.02,  # foreign interest rate
               'Cstar': 1,
               'Q': 1}  # Foreign demand

## Model setup

### `HetBlock` as before

We begin with the household block, which is defined exactly like we used it before! [copy & paste]

In [3]:
# initialize
def hh_init(a_grid, z, r, eis):
    coh = (1 + r) * a_grid[np.newaxis, :] + z[:, np.newaxis]
    Va = (1 + r) * (0.1 * coh) ** (-1 / eis)
    return Va

# backward step
@sj.het(exogenous='Pi',  # <-- this means our transition matrix will be fed into the model as Pi
        policy='a',  # <-- this means our endogenous state variable is a, defined over grid a_grid
        backward='Va',  # <-- this means we're iterating over variable Va, whose future value is Va_p
        backward_init=hh_init)
def hh(Va_p, a_grid, z, r, beta, eis):
    uc_nextgrid = beta * Va_p  # u'(c') on tomorrow's grid
    c_nextgrid = uc_nextgrid ** (-eis)  # c' on tomorrow's grid
    coh = (1 + r) * a_grid[np.newaxis, :] + z[:, np.newaxis]  # cash on hand coh on today's grid
    a = sj.interpolate.interpolate_y(c_nextgrid + a_grid, coh, a_grid)  # this plots (c'+a', a') pairs and computes policy a from interpolation on coh
    sj.misc.setmin(a, a_grid[0])  # impose borrowing constraint
    c = coh - a  # back out consumption
    Va = (1 + r) * c ** (-1 / eis)  # V'(a)
    return Va, a, c

Just like before, we set two `hetinputs` and a `hetoutput` for calculating MPCs. [copy & paste]

In [4]:
def make_grids(rho_e, sd_e, n_e, min_a, max_a, n_a):
    e_grid, _, Pi = sj.grids.markov_rouwenhorst(rho_e, sd_e, n_e)
    a_grid = sj.grids.asset_grid(min_a, max_a, n_a)
    return e_grid, Pi, a_grid

def income(Z, e_grid):
    z = Z * e_grid
    return z

def compute_weighted_mpc(c, a, a_grid, r, e_grid):
    """Approximate mpc out of wealth, with symmetric differences where possible, exactly setting mpc=1 for constrained agents."""
    mpc = np.empty_like(c)
    post_return = (1 + r) * a_grid
    mpc[:, 1:-1] = (c[:, 2:] - c[:, 0:-2]) / (post_return[2:] - post_return[:-2])
    mpc[:, 0] = (c[:, 1] - c[:, 0]) / (post_return[1] - post_return[0])
    mpc[:, -1] = (c[:, -1] - c[:, -2]) / (post_return[-1] - post_return[-2])
    mpc[a == a_grid[0]] = 1
    mpc = mpc * e_grid[:, np.newaxis]
    return mpc

hh_extended = hh.add_hetinputs([make_grids, income]).add_hetoutputs([compute_weighted_mpc])

### Other blocks

Here we list a few extra `SimpleBlock`s we need:

 - sectoral demand of domestic households:

 - foreign demand

 - UIP

 - income

 - balance of payments: (just to compute the NFA, not needed if we clear the goods market)

 - market clearing

We could allow for more here: Wage Phillips curve, Taylor rule, etc, but that is not necessary for what we are doing now, as long as we assume real interest rate rules.

## Steady state

## Calibration and model setup

We calibrate the model, just like before. First, we give it a test run, without calibrating anything.

Automate this ...

Now let's verify that Walras law and UIP hold:

What's `chi` here?

## Transitional dynamics

We are now going to look at the impulse responses to two kinds of shocks:
1. Shocks to the foreign interest rate, $r^*$, interpreted as exchange rate shocks or capital flow shocks.
2. Shocks to the domestic interest rate, $r$, interpreted as domestic monetary policy shocks.

As before, we define a simple function to aid with plotting stuff.

In [5]:
def show_irfs(irfs_list, variables, labels=[" "], ylabel=r"Percentage points (dev. from ss)", T_plot=50, figsize=(18, 6)):
    if len(irfs_list) != len(labels):
        labels = [" "] * len(irfs_list)
    n_var = len(variables)
    fig, ax = plt.subplots(1, n_var, figsize=figsize, sharex=True)
    for i in range(n_var):
        # plot all irfs
        for j, irf in enumerate(irfs_list):
            ax[i].plot(100 * irf[variables[i]][:50], label=labels[j])
        ax[i].set_title(variables[i])
        ax[i].set_xlabel(r"$t$")
        if i==0:
            ax[i].set_ylabel(ylabel)
        ax[i].legend()
    plt.show()

### Shocks to the foreign interest rate, $r^*$

We use `solve_impulse_linear` just like before.

Let's plot those:

Why does consumption not move here? Despite `Y` moving? Let us also plot `Z`, `PH_P`:

We see that real labor income Z is constant, because the real value of goods sold, `PH_P` falls due to the depreciation, exactly offsetting the increase in output `Y`.

We can learn more about the propagation by separating the `C` response into what would have happened with only `PH_P` falling ("real income channel") and with only the `Y` increase ("multiplier channel"). We do so by using the `impulse_linear` function.

We see that what's going on is that the real income channel and the multiplier channel are exactly offsetting each other here.

Let's see what happens with greater and smaller `chi`s:

Repeating the decomposition for the case with low chi:

Here the real income channel dominates!

### Shocks to the domestic interest rate, r

Let's try out a domestic monetary policy shock. For that, let's set chi to 2 - alpha.

Notice that `Q` responds exactly the same here as it did to a positive `rstar` shock. But `Y` responds more, because `C` responds more.

Why is that? Let's decompose the `C` response once more.

## Exercises

### Exercise 1: `Cstar` shocks

Compute the IRFs to an export demand shock, i.e. a shock to `Cstar`. Is there an effect on `C`? If so, why?

### Exercise 2: Domestic demand shocks

Compute the IRFs to a domestic demand shock, i.e. a shock to `beta`.

 (a) How different is a positive demand shock from an accommodative monetary policy shock?

 (b) Decompose the `C` response into the direct effect of the `beta` shock, and the indirect response via `Z`.
 
 (c) Simulate a shock to income risk, i.e. to `sd_e`. How different is a positive risk shock from a negative demand shock?

### Exercise 3: UIP shocks

(a) Modify the model to introduce an exogenous UIP wedge and hit it with a shock. Compare it to the effect of an `rstar` shock, suitably rescaled, with the same persistence.

(b) Now make the UIP wedge endogenous to the NFA: Pick a simple affine linear function for the UIP wedge. How does the endogenous UIP wedge affect the propagation of `rstar` and `r` shocks? What if `chi` is low?

### Exercise 4: MPCs

Add an option to compute MPCs, just like we did before, as `hetoutput`. Compute MPCs in this steady state. Have the model recalibrate the steady state markup `markup_ss` to hit a quarterly MPC of 0.25.

### Exercise 5: Comparison to RA and TA

Just like before, introduce RA and TA models. Compare the HA IRFs to the ones in the RA and TA models. Show that they coincide for `rstar` shocks when `chi=1` but can differ significantly otherwise.

### Exercise 6: Phillips curve and Taylor rule

(a) Introduce a standard wage Phillips curve into the model. Plot the response of nominal wage inflation to an `rstar` shock.

(b) Add a Taylor rule, which pins down the real rate as a function of domestic inflation. How does the response to `rstar` shocks change with a Taylor rule compared to a real rate rule? Interpret.